# 911 Calls - Data Analysis with Python

For this project we will be analyzing some 911 call data from [Kaggle](https://www.kaggle.com/mchirico/montcoalert). The data contains the following fields:

* lat : String variable, Latitude
* lng: String variable, Longitude
* desc: String variable, Description of the Emergency Call
* zip: String variable, Zipcode
* title: String variable, Title
* timeStamp: String variable, YYYY-MM-DD HH:MM:SS
* twp: String variable, Township
* addr: String variable, Address
* e: String variable, Dummy variable (always 1)

Let's Start

## Environment and Data Setup

In [1]:
import numpy as np
import pandas as pd

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

**Read in the csv file as a dataframe called df**

In [1]:
df = pd.read_csv('/kaggle/input/montcoalert/911.csv')

In [1]:
df.info()

**Check the head of df**

In [1]:
df.head(10)

##  Questions

**What are the top 5 zipcodes for 911 calls?**

In [1]:
df['zip'].value_counts().head(5)

**What are the top 5 townships (twp) for 911 calls?**

In [1]:
df['twp'].value_counts().head(5)

**Take a look at the 'title' column, how many unique title codes are there?**

In [1]:
df['title'].nunique()

## Creating new features

**In the titles column there are "Reasons/Departments" specified before the title code. These are EMS, Fire, and Traffic. Use .apply() with a custom lambda expression to create a new column called "Reason" that contains this string value.**

*For example, if the title column value is EMS: BACK PAINS/INJURY , the Reason column value would be EMS.*

In [1]:
df['Reason'] = df['title'].apply(lambda title: title.split(':')[0])

In [1]:
df.head()

In [1]:
df['Reason'].unique()

**What is the most common Reason for a 911 call based off of this new column?**

In [1]:
df['Reason'].value_counts()

**Now use seaborn to create a countplot of 911 calls by Reason.**

In [1]:
sns.countplot(x='Reason',data=df,palette='viridis')

In [1]:
df.info()

**Now let us begin to focus on time information. Timestamp is not in date time format. What is the data type of the objects in the timeStamp column?**

In [1]:
type(df['timeStamp'].iloc[0])

**These timestamps are still strings. convert the column from strings to DateTime objects.**

In [1]:
df['timeStamp'] = pd.to_datetime(df['timeStamp'])

In [1]:
type(df['timeStamp'].iloc[0])

In [1]:
df.head()

**Now that the timestamp column are actually DateTime objects, use .apply() to create 3 new columns called Hour, Month, and Day of Week.**

In [1]:
df['Hour'] = df['timeStamp'].apply(lambda time: time.hour)
df['Month'] = df['timeStamp'].apply(lambda time: time.month)
df['Day of Week'] = df['timeStamp'].apply(lambda time: time.dayofweek)

In [1]:
df.head()

**Notice how the Day of Week is an integer 0-6. Use the .map() with this dictionary to map the actual string names to the day of the week:**

dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

In [1]:
dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

In [1]:
df['Day of Week'] = df['Day of Week'].map(dmap)

In [1]:
df['Day of Week']

**Now use seaborn to create a countplot of the Day of Week column with the hue based off of the Reason column.**

In [1]:
sns.countplot(x='Day of Week',data=df,hue='Reason',palette='viridis')

# To relocate the legend
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

**Now do the same for Month**

In [1]:
sns.countplot(x='Month',data=df,hue='Reason',palette='viridis')

# To relocate the legend
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

**Now create a gropuby object called byMonth, where you group the DataFrame by the month column and use the count() method for aggregation. Use the head() method on this returned DataFrame.**

In [1]:
byMonth = df.groupby('Month').count()
byMonth

**Now create a simple plot off of the dataframe indicating the count of calls per month.**

In [1]:
# Could be any column
byMonth['twp'].plot()

**Now see if you can use seaborn's lmplot() to create a linear fit on the number of calls per month. Keep in mind you may need to reset the index to a column.**

In [1]:
sns.lmplot(x='Month',y='twp',data=byMonth.reset_index())

**Create a new column called 'Date' that contains the date from the timeStamp column. You'll need to use apply along with the .date() method.**

In [1]:
df['Date']=df['timeStamp'].apply(lambda t: t.date())

**Now groupby this Date column with the count() aggregate and create a plot of counts of 911 calls.**

In [1]:
df.groupby('Date').count()['twp'].plot()
plt.tight_layout()

**Now recreate this plot but create 3 separate plots with each plot representing a Reason for the 911 call**

In [1]:
df[df['Reason']=='Traffic'].groupby('Date').count()['twp'].plot()
plt.title('Reason Traffic')
plt.tight_layout()

In [1]:
df[df['Reason']=='Fire'].groupby('Date').count()['twp'].plot()
plt.title('Reason Fire')
plt.tight_layout()

In [1]:
df[df['Reason']=='EMS'].groupby('Date').count()['twp'].plot()
plt.title('Reason EMS')
plt.tight_layout()

**Now let's move on to creating heatmaps with seaborn and our data. We'll first need to restructure the dataframe so that the columns become the Hours and the Index becomes the Day of the Week**

In [1]:
dayHour = df.groupby(by=['Day of Week','Hour']).count()['Reason'].unstack()
dayHour.head()

**Now create a HeatMap using this new DataFrame**

In [1]:
plt.figure(figsize=(12,6))
sns.heatmap(dayHour,cmap='viridis')

**Now create a clustermap using this DataFrame.**

In [1]:
sns.clustermap(dayHour,cmap='viridis')

**Now repeat these same plots and operations, for a DataFrame that shows the Month as the column.**

In [1]:
dayMonth = df.groupby(by=['Day of Week','Month']).count()['Reason'].unstack()
dayMonth.head()

In [1]:
plt.figure(figsize=(12,6))
sns.heatmap(dayMonth,cmap='viridis')

In [1]:
sns.clustermap(dayMonth,cmap='viridis')